# Assignment 1: Exploring Zipf's Law

#### Due date: January 13, 11:59 PM

For this assignment, you will submit two files on Gradescope, `hw1_dist.ipynb` and `hw1_dist.pdf`. `hw1_dist.ipynb` goes under "Assignment 1 - Code" and `hw1_dist.pdf` goes under "Assignment 1 - Written."

### What is Zipf's Law?

Zipf's Law is an empirical observation that in any large corpus of natural language, the frequency of a word is inversely proportional to its frequency rank. Zipf's Law is a special case of a power law distribution.

More generally, the relationship between rank ($r$) and frequency ($f$) can be modeled as:
$$f \propto \frac{1}{r^s}$$
where $s$ is the Zipf exponent (for natural language, $s \approx 1$).

#### Log-Space Derivation
To see why this appears as a straight line on a log-log plot, we take the logarithm of both sides:
1. Start with the power law: $f = \frac{C}{r^s}$
2. Take the log: $\log(f) = \log\left(\frac{C}{r^s}\right)$
3. Use log properties: $\log(f) = \log(C) - \log(r^s)$
4. Final linear form: $\log(f) = \log(C) - s \cdot \log(r)$

Comparing this to the standard linear equation $y = mx + b$, where $y = \log(f)$ and $x = \log(r)$:
- The slope ($m$) is $-s$.
- The y-intercept ($b$) is $\log(C)$.

For natural language where $s \approx 1$, we expect a slope of approximately -1.0. This linear signature is a unique property of natural language and is notably absent in randomly generated text.

#### Relationship Between Zipf Exponent $s$ and Distribution Exponent $\alpha$

The Zipf exponent $s$ describes the rank-frequency relationship ($f \propto r^{-s}$). There is a related but distinct exponent $\alpha$ that describes the probability distribution of word frequencies: $P(f) \propto f^{-\alpha}$.

The relationship between them is: $\alpha = 1 + \frac{1}{s}$, or equivalently $s = \frac{1}{\alpha - 1}$.

For natural language where $s \approx 1$, we expect $\alpha \approx 2$.

#### Testing Power Law Fit
To rigorously test whether a distribution follows a power law, we use the method from Clauset, Shalizi, and Newman (2009), "Power-law distributions in empirical data" (https://arxiv.org/abs/0706.1062). This approach:

1. **Estimates $\alpha$**: Fits the power law exponent using maximum likelihood estimation.

2. **Compares alternatives**: Tests whether power law is a better fit than exponential or lognormal distributions using likelihood ratio tests.

3. **The $x_{min}$ parameter**: An important parameter is $x_{min}$, the minimum frequency value above which the power law is fitted. Words with very low frequencies often deviate from the power law pattern. In this exercise, we set $x_{min} = 5$. You can read the reference above for more details on how to choose $x_{min}$, and you can try other values in the assignment.

### Assignment Overview
In this assignment, we will provide an implementation for a basic version of the following workflows.
1. Tokenization: A function to return a list of words given a document.
2. Frequency Counting: Compute word occurrences across your corpus.
3. Visualization: Plot the frequency-rank relationship on a log-log scale.
4. Parameter Estimation: Estimate the distribution exponent $\alpha$, derive the Zipf exponent $s$, and test whether power law is supported.

Your task will be:
1. Run the pipeline on a human corpus and interpret the results.
2. Run the pipeline on an AI corpus and interpret the results.
3. Run the pipeline on a mystery corpus and explain the results.

### Main implementation

Use the functions in the below cell in your analyses.

In [1]:
import re
from collections import Counter
import matplotlib.pyplot as plt
import powerlaw
import numpy as np

def tokenize(text):
    return re.findall(r'\w+', text.lower())

def count_word_frequencies(text, tokenizer):
    tokens = tokenizer(text)
    frequencies = Counter(tokens)
    return frequencies

def fit_zipf(frequencies, xmin=5, plot=True):
    """
    Fit a power law to word frequencies and test goodness of fit.
    
    Uses the method from Clauset, Shalizi, and Newman (2009):
    "Power-law distributions in empirical data"
    https://arxiv.org/abs/0706.1062
    
    Args:
        frequencies: Dict or Counter mapping tokens to counts.
        xmin: Minimum frequency value for power law fitting (default=5).
        plot: Whether to plot the PDF with fitted power law.
    Returns:
        Dict with fit results including alpha, s, and comparison verdicts.
    """
    freq_values = np.array(list(frequencies.values()))
    
    # Fit power law distribution using Clauset et al. method
    fit = powerlaw.Fit(freq_values, xmin=xmin, discrete=True, verbose=False)
    
    # Derive Zipf exponent s from alpha: alpha = 1 + 1/s, so s = 1/(alpha - 1)
    alpha = fit.power_law.alpha
    s = 1 / (alpha - 1)
    
    # Compare power law to exponential distribution
    R_exp, p_exp = fit.distribution_compare('power_law', 'exponential', normalized_ratio=True)
    
    # Compare power law to lognormal distribution
    R_log, p_log = fit.distribution_compare('power_law', 'lognormal', normalized_ratio=True)
    
    # Determine verdict for each comparison
    def get_verdict(R, p, alt_name):
        if p >= 0.05:
            return f"indistinguishable from {alt_name}"
        elif R > 0:
            return f"power law preferred over {alt_name}"
        else:
            return f"{alt_name} preferred over power law"
    
    verdict_exp = get_verdict(R_exp, p_exp, "exponential")
    verdict_log = get_verdict(R_log, p_log, "lognormal")
    
    result = {
        'alpha': alpha,
        's': s,
        'xmin': xmin,
        'R_vs_exponential': R_exp,
        'p_vs_exponential': p_exp,
        'R_vs_lognormal': R_log,
        'p_vs_lognormal': p_log,
        'verdict_exponential': verdict_exp,
        'verdict_lognormal': verdict_log,
    }
    
    print(f"alpha: {alpha:.3f} (xmin={xmin})")
    print(f"Zipf exponent s: {s:.3f} (derived from alpha)")
    print(f"vs Exponential: R={R_exp:.3f}, p={p_exp:.3f} -> {verdict_exp}")
    print(f"vs Lognormal:   R={R_log:.3f}, p={p_log:.3f} -> {verdict_log}")
    
    if plot:
        fig, ax = plt.subplots()
        fit.plot_pdf(ax=ax, color='b', label='Data')
        fit.power_law.plot_pdf(ax=ax, color='r', linestyle='--', label='Power law fit')
        ax.set_xlabel('Frequency')
        ax.set_ylabel('PDF')
        ax.set_title("Zipf's Law")
        ax.legend()
        plt.show()
    
    return result

def zipf_law_main(documents, xmin=5, tokenizer=tokenize, plot=True):
    frequencies = count_word_frequencies(documents, tokenizer)
    result = fit_zipf(frequencies, xmin=xmin, plot=plot)
    return result, frequencies

### 1. Experiment with Moby Dick

For this exercise, we will analyze **Moby Dick** from the NLTK Gutenberg corpus. Run the given code and answer the following questions.

(a) Plot the Zipf's law graph for the corpus. What is the estimated $s$? Is power law supported? (10 points)

(b) How many words are there in the corpus? What is the vocabulary size? What are the 20 most common and least common words? (20 points)

In [ ]:
print("Loading Moby Dick from moby_dick.txt...")
with open("./data/moby_dick.txt", "r") as fw:
    moby_dick = fw.read()[:950000]

print(f"Loaded Moby Dick.", moby_dick[:100])

In [ ]:
### 1a. YOUR CODE HERE

In [ ]:
### 1b. YOUR CODE HERE
# Put your results in the variables below

total_words_moby_dick = #
vocab_size_moby_dick = #

top_20_moby_dick = #
least_20_moby_dick = #

### 2. Experiment with an AI-generated corpus

Next, we will analyze an AI-generated corpus and compare it against *Moby Dick* (`data/ai_novel.txt`). The corpus contains all of the books in `GOAT-AI/generated-novels` on HuggingFace. It was minimally processed to split words concatenated together due to PDF extraction errors.

(a) Plot the Zipf's Law graph and report the fitted exponent $s$ for the AI corpus. How does it compare to *Moby Dick*? (10 points)

(b) How many words are there in the corpus? What is the vocabulary size? What are the 20 most common and least common words? (20 points)

In [ ]:
print("Loading AI novel from moby_dick.txt...")
with open("./data/ai_novel.txt", "r") as handle:
    ai_novel = handle.read()

print(f"Loaded AI novel.", ai_novel[:100])

In [ ]:
### 2a. YOUR CODE HERE

In [ ]:
### 2b. YOUR CODE HERE
# Put your results in the variables below

total_words_ai_novel = #
vocab_size_ai_novel = #

# Get the 20 most common and least common words
top_20_ai_novel = #
least_20_ai_novel = #

### 3. Examine the differences between the two corpora and explain why the observations are different.

(a) Compute the following quantities for the two corpora:
- `total_words`: all words in the corpus
- `vocab_size`: the number of unique words
- `type_token_ratio`: `vocab_size / total_words`
- `hapax_ratio`: hapax legomena, fraction of words that appear exactly once
- `top10_coverage`: fraction of tokens covered by the top 10 words
- `top100_coverage`: fraction of tokens covered by the top 100 words
- `top1000_coverage`: fraction of tokens covered by the top 1000 words
- `top2000_coverage`: fraction of tokens covered by the top 2000 words

Optional: you are encouraged to perform more analyses for comparison. For example, you can plot the two Zipf's Law lines on the same plot and look at the visual differences. (10 points)

(b) Based on your calculations in part (a), produce a short write-up comparing Moby Dick and the AI corpus. Put your final answer in the `Answer` cell below. (10 points)


In [ ]:
### 3a. YOUR CODE HERE

moby_dick_stats = {
    'total_words': ,
    'vocab_size': ,
    'type_token_ratio': ,
    'hapax_ratio': ,
    'top10_coverage': ,
    'top100_coverage': ,
    'top1000_coverage': ,
    'top2000_coverage': ,
}

ai_novel_stats = {
    'total_words': ,
    'vocab_size': ,
    'type_token_ratio': ,
    'hapax_ratio': ,
    'top10_coverage': ,
    'top100_coverage': ,
    'top1000_coverage': ,
    'top2000_coverage': ,
}


#### Answer

### 4. Analyzing a mystery corpus

You are given a mystery corpus that does not follow Zipf's Law. Investigate this corpus and give a reason for why it deviates from Zipf's Law. Put your answer in the `Answer` cell below. (20 points)

Hint: you should explore ways to look at the data.

In [ ]:
print("Loading mystery corpus from mystery.txt...")
with open("./data/mystery.txt", "r") as fin:
    mystery = fin.read()

print(f"Loaded mystery corpus: ", mystery[:100])

In [ ]:
### 4. YOUR CODE HERE

#### Answer